In [61]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import MSELoss
from torch.nn.init import xavier_uniform_

from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston

import numpy as np
from numpy import vstack
from numpy import sqrt

In [62]:
class inputDataset(Dataset):
    # load the dataset
    def __init__(self):
        # store the inputs and outputs
        self.X, self.y = load_boston(return_X_y=True)
        self.X, self.y = self.X.astype('float32'), self.y.astype('float32')
        # ensure target has the right shape
        self.y = self.y.reshape((len(self.y), 1))
 
    # number of rows in the dataset
    def __len__(self):
        return len(self.X)
 
    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]
 
    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])
    
    def prepare_data(self):
        # calculate split
        train, test = self.get_splits()
        # prepare data loaders
        train_dl = DataLoader(train, batch_size=32, shuffle=True)
        test_dl = DataLoader(test, batch_size=1024, shuffle=False)
        return train_dl, test_dl

In [63]:
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 10)
        xavier_uniform_(self.hidden1.weight)
        self.act1 = Sigmoid()
        # second hidden layer
        self.hidden2 = Linear(10, 8)
        xavier_uniform_(self.hidden2.weight)
        self.act2 = Sigmoid()
        # third hidden layer and output
        self.hidden3 = Linear(8, 1)
        xavier_uniform_(self.hidden3.weight)
 
    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        return X

In [64]:
class Model:
    
    def __init__(self):
        self.model = MLP(13)
    
    def train(self, train_dl):
        # define the optimization
        criterion = MSELoss()
        optimizer = SGD(self.model.parameters(), lr=0.01, momentum=0.9)
        # enumerate epochs
        for epoch in range(100):
            # enumerate mini batches
            for i, (inputs, targets) in enumerate(train_dl):
                # clear the gradients
                optimizer.zero_grad()
                # compute the model output
                yhat = self.model(inputs)
                # calculate loss
                loss = criterion(yhat, targets)
                # credit assignment
                loss.backward()
                # update model weights
                optimizer.step()
    
    def test(self, test_dl):
        predictions, actuals = list(), list()
        for i, (inputs, targets) in enumerate(test_dl):
            # evaluate the model on the test set
            yhat = self.model(inputs)
            # retrieve numpy array
            yhat = yhat.detach().numpy()
            actual = targets.numpy()
            actual = actual.reshape((len(actual), 1))
            # store
            predictions.append(yhat)
            actuals.append(actual)
        predictions, actuals = vstack(predictions), vstack(actuals)
        # calculate mse
        mse = mean_squared_error(actuals, predictions)
        return mse
    
    def predict(self, row):
        # convert row to data
        row = Tensor([row])
        # make prediction
        yhat = self.model(row)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        return yhat

In [65]:
model = Model()

In [66]:
train_dl, test_dl = inputDataset().prepare_data()

In [67]:
model.train(train_dl)

In [68]:
model.test(test_dl)

70.51213

In [69]:
row = [0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,396.90,4.98]
model.predict(row)

array([[22.618038]], dtype=float32)